# Talk with any Document using ChatCompletion API, Embeddings, and Pinecone

In this project, we will build a highly interactive and intelligent system that lets users 'talk' with any document. Leveraging the capabilities of OpenAI's ChatCompletion API, the semantic understanding of embeddings, we'll create an application that can understand and retrieve information from documents in a conversational manner. For demonstration purposes, I have included Finance Minister's 2025 Budget speech as the reference document and asking questions related to this document.

## Objectives

We will develop a system that can:

1. **Interpret Documents**: Analyze and understand the content of various documents through the power of embeddings.
2. **Conversational Interface**: Provide users with the ability to ask questions and receive answers as if they were talking to a human expert on the document's content.
3. **Contextual Awareness**: Maintain the thread of conversation, taking into account previous interactions and the document's subject matter.
4. **Scalable Document Handling**: Efficiently manage and query a large number of documents using Pinecone's vector database capabilities.
5. **Integrate RAG**: Retrieval-augmented generation (RAG) for large language models (LLMs) aims to improve prediction quality by using an external datastore at inference time to build a richer prompt that includes some combination of context, history, and recent/relevant knowledge.


# 2. Libraries import

In [ ]:
!pip install openai
!pip install python-dotenv

In [ ]:
!pip install PyPDF2
!pip install pinecone-client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 4.7 MB/s eta 0:00:00


In [ ]:
import os
import openai
import PyPDF2
import random
import pinecone

from openai import OpenAI
from dotenv import load_dotenv

# 3. Working with PDF files

In this section, we will read in a pdf document, split it into chunks, convert the chunks to enbeddings and store these embeddings in Pinecone vector database. Later these embeddings will be used to perform a similarity search and retrieve relevant embeddings.



### 3.1 Setting up API Key

We will read in the API key from the .env file and check if it has been loaded correctly.

In [ ]:
# os.environ["OPENAI_API_KEY"] = "sk-XXXXXXXXXXXXX"
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")

client = OpenAI()

API key looks good so far


### 3.2 Loading a PDF file




In [ ]:
# Function to load a random PDF from a given directory
def load_pdf(file_name):
  _file = open(file_name, 'rb')
  pdf_reader = PyPDF2.PdfReader(_file)

  text_from_pdf = ''
  for page in range(len(pdf_reader.pages)):
    text_from_pdf += pdf_reader.pages[page].extract_text()
  return text_from_pdf


In [ ]:
# Function to chunk text by number of words or characters with a given size and overlap
def chunk_text(text, chunk_size=1500, chunk_overlap=100, by='word'):
    if by not in ['word', 'char']:
        raise ValueError("Invalid value for 'by'. Use 'word' or 'char'.")

    chunks = []

    if by == 'word':
        text = text.split()
    elif by == 'char':
        text = text

    current_chunk_start = 0
    while current_chunk_start < len(text):
        current_chunk_end = current_chunk_start + chunk_size

        if by == 'word':
            chunk = ' '.join(text[current_chunk_start:current_chunk_end])
        elif by == 'char':
            chunk = text[current_chunk_start:current_chunk_end]

        chunks.append(chunk)
        current_chunk_start += (chunk_size - chunk_overlap)

    return chunks

In [ ]:
pdf_loaded = load_pdf("budget_speech.pdf")

In [ ]:
#Display the first 1000 chunks.
pdf_loaded[:1000]

'GOVERNMENT OF INDIA\nBUDGET 2025-2026\nSPEECH\nOF\nNIRMALA SITHARAMAN\nMINISTER OF FINANCE\nFebruary 1,  2025 \nCONTENTS  \n \nPART – A \n Page No.  \nIntroduction  1 \nBudget Theme  1 \nAgriculture as the 1st engine  3 \nMSMEs as the 2nd engine  6 \nInvestment as the 3rd engine  8 \nA. Investing in People  8 \nB. Investing in  the Economy  10 \nC. Investing in Innovation  14 \nExports as the 4th engine  15 \nReforms as the Fuel  16 \nFiscal Policy  18 \n \n \nPART – B \nIndirect taxes  20 \nDirect Taxes   23 \n \nAnnexure to Part -A 29 \nAnnexure to Part -B 31 \n \n   \n \nBudget 202 5-2026 \n \nSpeech of  \nNirmala Sitharaman  \nMinister of Finance  \nFebruary 1 , 202 5 \nHon’ble Speaker,  \n I present the Budget for 2025 -26. \nIntroduction  \n1. This Budget continues our Government ’s efforts to:  \na) accelerate growth,  \nb) secure inclusive development,  \nc) invigorate private sector investments,  \nd) uplift household sentiments, and \ne) enhance spending power of India’s ris

In [ ]:
chunks = chunk_text(pdf_loaded, by='char')

In [ ]:
len(chunks)

67

## 4. Building RAG system (Retrieval Augmented System)

First an empty database is created in Pinecone at the Pinecone website. Then the embeddings are inserted into the empty Pinecone database. This section assumes that an empty database called 'budget' has already been created in Pinecone.

In [ ]:
# Pinecone init
from pinecone import Pinecone
pc = Pinecone(api_key=os.getenv('PINECONE_API_KEY'))


In [ ]:
index = pc.Index('budget')

In [ ]:
for i in range(len(chunks)):
  vector = client.embeddings.create(
    input=chunks[i],
    model="text-embedding-ada-002"
  )

  insert_stats = index.upsert(
      vectors = [
          (
              str(i),
              vector.data[0].embedding,
              {'org_text': chunks[i]}
          )
      ]
  )

print(vector.data[0].embedding)
print(insert_stats)


[-0.02033730037510395, -0.004234557505697012, 0.007862137630581856, -0.024479025974869728, -0.012503726407885551, -0.005469934083521366, -0.01846638321876526, -0.0296490415930748, -0.032305460423231125, -0.009333164431154728, 0.021279899403452873, 0.022950872778892517, 0.007990674115717411, 0.014981620945036411, 0.0021958285942673683, 0.011853904463350773, 0.015652867034077644, -0.0027938794810324907, 0.020522965118288994, -0.024250516667962074, -0.011868186295032501, -0.015395794063806534, -0.0487295426428318, -0.008269169367849827, -0.008597650565207005, 0.0002327489055460319, 0.036332931369543076, -0.02589292638003826, -0.011653958819806576, -0.00085646245861426, -0.003493688302114606, -0.004270262084901333, -0.023664962500333786, -0.008761892095208168, -0.028192296624183655, -0.005162875168025494, 0.002336861565709114, -0.007762164808809757, 0.024036290124058723, -0.0020190912764519453, 0.019666055217385292, 0.008840441703796387, -0.01277508120983839, 0.00997584592550993, -0.002335

### 5. Building an interface to get proper answer based on the documentation

Here a simple user interface is created for getting a user's input question and that question is converted into embedding using the same embedding model that was used to convert the pdf into embedding. These embeddings are then passed to the Pinecone database and top n matching embedddings are retrieved.


In [ ]:
user_input = input("Ask a question about the budget: ")

#print(user_input)

user_vector = client.embeddings.create(
    model = 'text-embedding-ada-002',
    input = user_input
)

user_vector = user_vector.data[0].embedding

matches = index.query(
    vector=user_vector,
    top_k=5,
    include_metadata=True
)

print(matches['matches'][0]['metadata']['org_text'])
print(matches['matches'][1]['metadata']['org_text'])
print(matches['matches'][2]['metadata']['org_text'])
print(matches['matches'][3]['metadata']['org_text'])
print(matches['matches'][4]['metadata']['org_text'])

Ask a question about the budget: Calculate the tax applicable on an income of 1714748 as per new tax proposals.
ital gains ) under the new regime. This limit will be ` 12.75 
lakh for salaried tax payers, due to standard deduction of ` 75,000.  
157. Slabs and rates are being changed across the board to benefit all tax -
payers. The new structure will substantially reduce the taxes of the middle 
class and leave more money in their hands, boosting household consumption, 
savings and investment.  
158.  In the new tax regime, I propose to revise tax rate structure as follows:  
0-4 lakh rupees  Nil 
4-8 lakh rupees  5 per cent  
8-12 lakh rupees  10 per cent  
12-16 lakh rupees  15 per cent  
16-20 lakh rupees  20 per cent  
20- 24 lakh rupees  25 per cent  
Above 24 lakh rupees  30 per cent   28  
 
159. To tax payers upto ` 12 lakh of normal income (other than special rate 
income  such as capital gains ) tax rebate is being provided in addition to the 
benefit due to slab rate reduct

The text from retrieved appendings along with the user input is then passed to Chat completions api along witha relevant system message. The response is then collected from the text returned by the chat completions api.

In [ ]:
messages = [{"role": "developer", "content": '''I want you to act as an expert financial analyst. Your name is "FinWiz Assistant". You will provide me with answers from the given info. Always think step by step and then only answer the question. Carefully match the tax slabs while calculating. If the answer is not included, say exactly "Question outside my scope of assisstance." and stop after that. Always stick to your character.'''}]
messages.append({"role": "user", "content": user_input})
messages.append({"role": "user", "content": matches['matches'][0]['metadata']['org_text']})
messages.append({"role": "user", "content": matches['matches'][1]['metadata']['org_text']})
messages.append({"role": "user", "content": matches['matches'][2]['metadata']['org_text']})
messages.append({"role": "user", "content": matches['matches'][3]['metadata']['org_text']})
messages.append({"role": "user", "content": matches['matches'][4]['metadata']['org_text']})

#print(messages)
print("Question: " + user_input)

chat_messages = client.chat.completions.create(
    #model="gpt-3.5-turbo-0125",
    model="gpt-4o-mini",
    messages=messages,
    temperature=0,
    max_tokens=1000,
    stream=True
)

#print(chat_messages.choices[0].message.content)
# Iterate through the stream to get the complete response
full_response = 'Answer: '
for chunk in chat_messages:
    # Access the content from the 'delta' field within the chunk
    full_response += chunk.choices[0].delta.content or ''

print(full_response) # Print the assembled response

Question: Calculate the tax applicable on an income of 1714748 as per new tax proposals.
Answer: To calculate the tax applicable on an income of ₹1,714,748 as per the new tax proposals, we will follow these steps:

1. **Identify the applicable tax slabs**:
   - 0 - 4 lakh: Nil
   - 4 - 8 lakh: 5%
   - 8 - 12 lakh: 10%
   - 12 - 16 lakh: 15%
   - 16 - 20 lakh: 20%
   - 20 - 24 lakh: 25%
   - Above 24 lakh: 30%

2. **Break down the income into slabs**:
   - Income: ₹1,714,748
   - This falls into the 16 - 20 lakh slab.

3. **Calculate tax for each slab**:
   - For the first ₹4,00,000: Nil
   - For the next ₹4,00,000 (4,00,001 to 8,00,000): 5% of ₹4,00,000 = ₹20,000
   - For the next ₹4,00,000 (8,00,001 to 12,00,000): 10% of ₹4,00,000 = ₹40,000
   - For the next ₹4,00,000 (12,00,001 to 16,00,000): 15% of ₹4,00,000 = ₹60,000
   - For the next ₹4,00,000 (16,00,001 to 20,00,000): 20% of ₹1,71,748 = ₹34,350 (approximately)

4. **Total tax calculation**:
   - Total tax = ₹20,000 + ₹40,000 + ₹6

Again various questions pertaining to the document are asked.

In [ ]:
user_input = input("Ask a question about the budget: ")

user_vector = client.embeddings.create(
    model = 'text-embedding-ada-002',
    input = user_input
)

user_vector = user_vector.data[0].embedding

matches = index.query(
    vector=user_vector,
    top_k=5,
    include_metadata=True
)

print(matches['matches'][0]['metadata']['org_text'])
print(matches['matches'][1]['metadata']['org_text'])
print(matches['matches'][2]['metadata']['org_text'])
print(matches['matches'][3]['metadata']['org_text'])
print(matches['matches'][4]['metadata']['org_text'])

Ask a question about the budget: Calculate the tax applicable on a salaried income of 1150000 under the new tax regime
ital gains ) under the new regime. This limit will be ` 12.75 
lakh for salaried tax payers, due to standard deduction of ` 75,000.  
157. Slabs and rates are being changed across the board to benefit all tax -
payers. The new structure will substantially reduce the taxes of the middle 
class and leave more money in their hands, boosting household consumption, 
savings and investment.  
158.  In the new tax regime, I propose to revise tax rate structure as follows:  
0-4 lakh rupees  Nil 
4-8 lakh rupees  5 per cent  
8-12 lakh rupees  10 per cent  
12-16 lakh rupees  15 per cent  
16-20 lakh rupees  20 per cent  
20- 24 lakh rupees  25 per cent  
Above 24 lakh rupees  30 per cent   28  
 
159. To tax payers upto ` 12 lakh of normal income (other than special rate 
income  such as capital gains ) tax rebate is being provided in addition to the 
benefit due to slab rate

In [ ]:
messages = [{"role": "system", "content": '''I want you to act as an expert financial analyst. Your name is "FinWiz Assistant". You will provide me with answers from the given info. Always think step by step and then only answer the question. Carefully match the tax slabs while calculating. If the answer is not included, say exactly "Question outside my scope of assisstance." and stop after that. Always stick to your character.'''}]
messages.append({"role": "user", "content": user_input})
messages.append({"role": "user", "content": matches['matches'][0]['metadata']['org_text']})
messages.append({"role": "user", "content": matches['matches'][1]['metadata']['org_text']})
messages.append({"role": "user", "content": matches['matches'][2]['metadata']['org_text']})
messages.append({"role": "user", "content": matches['matches'][3]['metadata']['org_text']})
messages.append({"role": "user", "content": matches['matches'][4]['metadata']['org_text']})

#print(messages)
print("Question: " + user_input)

chat_messages = client.chat.completions.create(
    #model="gpt-3.5-turbo-0125",
    model="gpt-4o-mini",
    messages=messages,
    temperature=0,
    max_tokens=1000,
    stream=True
)

full_response = 'Answer: '
for chunk in chat_messages:
    # Access the content from the 'delta' field within the chunk
    full_response += chunk.choices[0].delta.content or ''

print(full_response) # Print the assembled response

Question: Calculate the tax applicable on a salaried income of 1150000 under the new tax regime
Answer: To calculate the tax applicable on a salaried income of ₹1,150,000 under the new tax regime, we first need to consider the applicable tax slabs:

1. **Income up to ₹4,00,000**: Nil
2. **Income from ₹4,00,001 to ₹8,00,000**: 5%
3. **Income from ₹8,00,001 to ₹12,00,000**: 10%
4. **Income from ₹12,00,001 to ₹16,00,000**: 15%
5. **Income from ₹16,00,001 to ₹20,00,000**: 20%
6. **Income from ₹20,00,001 to ₹24,00,000**: 25%
7. **Income above ₹24,00,000**: 30%

Given the income of ₹1,150,000, we first apply the standard deduction of ₹75,000 for salaried taxpayers, which brings the taxable income down to:

**Taxable Income = ₹1,150,000 - ₹75,000 = ₹1,075,000**

Now, we will calculate the tax based on the slabs:

- **First ₹4,00,000**: Nil
- **Next ₹4,00,000 (₹4,00,001 to ₹8,00,000)**: 5% of ₹4,00,000 = ₹20,000
- **Next ₹2,75,000 (₹8,00,001 to ₹10,75,000)**: 10% of ₹2,75,000 = ₹27,500

Now, w

In [ ]:
user_input = input("Ask a question about the budget: ")

print(user_input)

user_vector = client.embeddings.create(
    model = 'text-embedding-ada-002',
    input = user_input
)

user_vector = user_vector.data[0].embedding

matches = index.query(
    vector=user_vector,
    top_k=3,
    include_metadata=True
)

print(matches['matches'][0]['metadata']['org_text'])
print(matches['matches'][1]['metadata']['org_text'])
print(matches['matches'][2]['metadata']['org_text'])


Ask a question about the budget: What is the new limit of FDI in the insurance sector?
What is the new limit of FDI in the insurance sector?
o ‘Reforms as the Fuel’, and detail specific reforms.  
Tax Reforms  
94. Over the past 10 years, our Government  has implemented several 
reforms for convenience of tax payers, such as (1) faceless assessment, (2) tax 
payers charter, (3) faster returns, (4) almost 99 per cent returns being on self -
assessment, and (5) Vivad se Vishwas scheme. Continuing these efforts,   
I reaffirm the commitment of the tax department to “trust first, scrutinize later”. 
I also propose to introduce the new income -tax bill next week. I will detail the 
indirect tax reforms and changes in direct taxes in Part B.   
Financial Sector Reforms and Development  
FDI in Insurance Sector  
95. The FDI limit for the insurance sector will be raised from 74 to 100 per 
cent. This enhanced limit will be available for those companies which invest the 
entire premium in Indi

In [ ]:
messages = [{"role": "system", "content": '''I want you to act as an expert financial analyst. Your name is "FinWiz Assistant". You will provide me with answers from the given info. Always think step by step and then only answer the question. Carefully match the tax slabs while calculating.If the answer is not included, say "Question outside my scope of assisstance." '''}]
messages.append({"role": "user", "content": user_input})
messages.append({"role": "user", "content": matches['matches'][0]['metadata']['org_text']})
messages.append({"role": "user", "content": matches['matches'][1]['metadata']['org_text']})
messages.append({"role": "user", "content": matches['matches'][2]['metadata']['org_text']})

print("Question: " + user_input)

chat_messages = client.chat.completions.create(
    #model="gpt-3.5-turbo-0125",
    model="gpt-4o-mini",
    messages=messages,
    temperature=0,
    max_tokens=1000,
    stream=True
)

#print(chat_messages.choices[0].message.content)
# Iterate through the stream to get the complete response
full_response = 'Answer: '
for chunk in chat_messages:
    # Access the content from the 'delta' field within the chunk
    full_response += chunk.choices[0].delta.content or ''

print(full_response) # Print the assembled response

Question: What is the new limit of FDI in the insurance sector?
Answer: The new limit of Foreign Direct Investment (FDI) in the insurance sector has been raised from 74% to 100%. This enhanced limit will be available for those companies that invest the entire premium in India.


In [ ]:
user_input = input("Ask a question about the budget: ")

#print(user_input)

user_vector = client.embeddings.create(
    model = 'text-embedding-ada-002',
    input = user_input
)

user_vector = user_vector.data[0].embedding

matches = index.query(
    vector=user_vector,
    top_k=3,
    include_metadata=True
)

print(matches['matches'][0]['metadata']['org_text'])
print(matches['matches'][1]['metadata']['org_text'])
print(matches['matches'][2]['metadata']['org_text'])

Ask a question about the budget: What is Gyan Bharatam Mission according to the budget?
ve work we have done during our Government ’s first two terms 
guides us, to march forward resolutely.  
Budget Theme  
4. Our economy is the fastest -growing among all major global economies. 
Our development track record of the past 10 years and structural reforms have 
drawn global attention. Confidence in India’s capability and potential has only  2  
 
grown in this period. We see the next five years as a unique opportunity to 
realize ‘Sabka Vikas’, stimulating balanced growth of all regions.  
5. The great Telugu poet and playwright Gurajada Appa Rao had said, 
‘Desamante Matti Kaadoi, Desamante Manushuloi ’; meaning, ‘A country is not 
just its soil, a country is its people.’ In line with this, for us, Viksit Bharat, 
encompasses:  
a) zero -poverty;  
b) hundred per cent good quality school education;   
c) access to high -quality, affordable, and comprehensive healthcare;  
d) hundred per 

In [ ]:
messages = [{"role": "system", "content": '''I want you to act as an expert financial analyst. Your name is "FinWiz Assistant". You will provide me with answers from the given info. Always think step by step and then only answer the question. Carefully match the tax slabs while calculating. If a matching answer is not found, say "Question outside of my scope". '''}]
messages.append({"role": "user", "content": user_input})
messages.append({"role": "user", "content": matches['matches'][0]['metadata']['org_text']})
messages.append({"role": "user", "content": matches['matches'][1]['metadata']['org_text']})
messages.append({"role": "user", "content": matches['matches'][2]['metadata']['org_text']})

print("Question: " + user_input)

chat_messages = client.chat.completions.create(
    #model="gpt-3.5-turbo-0125",
    model="gpt-4o-mini",
    messages=messages,
    temperature=0,
    max_tokens=1000,
    stream=True
)

#print(chat_messages.choices[0].message.content)
# Iterate through the stream to get the complete response
full_response = 'Answer: '
for chunk in chat_messages:
    # Access the content from the 'delta' field within the chunk
    full_response += chunk.choices[0].delta.content or ''

print(full_response) # Print the assembled response

Question: What is Gyan Bharatam Mission according to the budget?
Answer: The Gyan Bharatam Mission, as outlined in the budget, focuses on the survey, documentation, and conservation of India's manuscript heritage. It aims to collaborate with academic institutions, museums, libraries, and private collectors to cover more than 1 crore manuscripts. Additionally, the mission will establish a National Digital Repository of Indian knowledge systems to facilitate knowledge sharing.


Some questions not relevant to the document are also asked to see the chatbot's response.

In [ ]:
user_input = input("Ask a question about the budget: ")

#print(user_input)

user_vector = client.embeddings.create(
    model = 'text-embedding-ada-002',
    input = user_input
)

user_vector = user_vector.data[0].embedding

matches = index.query(
    vector=user_vector,
    top_k=1,
    include_metadata=True
)

print(matches['matches'][0]['metadata']['org_text'])


Ask a question about the budget: What is the capital of India?
ig workers of online platforms provide great dynamism to the new -
age services economy.  Recognising their contribution, our Government  will 
arrange for their identity cards and registration on the e -Shram portal. They 
will be provided healthcare under PM Jan Arogya Yojana. This measure is likely 
to assist nearly 1 crore gig -workers.   
B. Investing in the Economy  
Public Private Partnership in Infrastructure  
52. Each infrastructure -related ministry will come up with a 3 -year pipeline 
of projects that can be implemented in PPP mode. States will also be 
encouraged to do so and can seek support from the IIPDF (India Infrastructure 
Project Development Fund) scheme to prepare PPP proposals.     
Support to States for Infrastructure  
53. An outlay of ` 1.5 lakh crore is proposed for the 50 -year interest free 
loans to states for capital expenditure and incentives for reforms.    
Asset Monetization Plan 2025 -30

In [ ]:
messages = [{"role": "system", "content": '''I want you to act as an expert financial analyst. Your name is "FinWiz Assistant". You will provide me with answers from the given info. Always think step by step and then only answer the question. Carefully match the tax slabs while calculating. If a matching answer is not found, say "Question outside of my scope". "'''}]
messages.append({"role": "user", "content": user_input})
messages.append({"role": "user", "content": matches['matches'][0]['metadata']['org_text']})


print("Question: " + user_input)

chat_messages = client.chat.completions.create(
    #model="gpt-3.5-turbo-0125",
    model="gpt-4o-mini",
    messages=messages,
    temperature=0,
    max_tokens=1000,
    stream=True
)

#print(chat_messages.choices[0].message.content)
# Iterate through the stream to get the complete response
full_response = 'Answer: '
for chunk in chat_messages:
    # Access the content from the 'delta' field within the chunk
    full_response += chunk.choices[0].delta.content or ''

print(full_response) # Print the assembled response

Question: What is the capital of India?
Answer: Question outside of my scope.
